<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Overlap" data-toc-modified-id="Overlap-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Overlap</a></span></li></ul></li><li><span><a href="#Host-tree" data-toc-modified-id="Host-tree-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Host tree</a></span><ul class="toc-item"><li><span><a href="#Filtering" data-toc-modified-id="Filtering-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Filtering</a></span></li><li><span><a href="#Writing-host-tree" data-toc-modified-id="Writing-host-tree-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Writing host tree</a></span></li></ul></li><li><span><a href="#Formatting-for-iTOL" data-toc-modified-id="Formatting-for-iTOL-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Formatting for iTOL</a></span><ul class="toc-item"><li><span><a href="#Copies-per-g-feces" data-toc-modified-id="Copies-per-g-feces-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Copies per g feces</a></span></li><li><span><a href="#Input-gDNA" data-toc-modified-id="Input-gDNA-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Input gDNA</a></span><ul class="toc-item"><li><span><a href="#color-strip" data-toc-modified-id="color-strip-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>color strip</a></span></li><li><span><a href="#heatmap" data-toc-modified-id="heatmap-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>heatmap</a></span></li></ul></li><li><span><a href="#Host-diet" data-toc-modified-id="Host-diet-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Host diet</a></span></li><li><span><a href="#Host-body-temp" data-toc-modified-id="Host-body-temp-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Host body temp</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Map qPCR results to the host phylogeny

# Var

In [3]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR/'

# processed qPCR data
qPCR_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR/2021-05-11/qPCR_results.tsv'

# host tree
host_tree_file = '/ebio/abt3_projects/Georg_animal_feces/data/animal/phylogeny/TT/unified_metadata_complete_20190416_rn_IndD.nwk'

# all sample metadata
metadata_file = '/ebio/abt3_projects/Georg_animal_feces/data/mapping/unified_metadata_complete_200429.tsv'

# body temp data
bt_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/bodyTemp_all.tsv'

# params
threads = 4

# Init

In [4]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(ape)
library(LeyLabRMisc)

In [5]:
make_dir(work_dir)
setDTthreads(threads)
df.dims()

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR/ 


# Load

In [12]:
# sample metadata
meta = Fread(metadata_file)
meta

SampleID,host_subject_id,host_german_name,host_common_name,scientific_name,scientific_name_subspecies,host_taxid,class,order,family,⋯,temp_regulation,collection_date,season,samp_store_days,samp_collect_device,sampling_method,diet,feeding_style,digestive_system,digestive_system_detail
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
F14_Common_Bream,F14,Brachse,Common.Bream,Abramis_brama,NA,38527,Actinopterygii,Cypriniformes,Cyprinidae,⋯,ectotherm/poikilotherm,8/10/10,summer,317,Greiner Polypropylene Tube 50ml,feces sampling,omnivore,NA,simple,simple
F35_Red_Deer,F35,Rothirsch,Red.Deer,Cervus_elaphus,NA,9860,Mammalia,Artiodactyla,Cervidae,⋯,endotherm/homoiotherm,9/29/10,autumn,229,Greiner Polystyrene 30ml Faeces Container,feces sampling,herbivore,intermediate type,foregut,foregut
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X189_fail_Rainbow_Trout,189.fail,Regenbogenforelle,Rainbow.Trout,Oncorhynchus_mykiss,NA,8022,Actinopterygii,Salmoniformes,Salmonidae,⋯,ectotherm/poikilotherm,3/1/12,winter,467,Eppendorf tube,feces sampling,carnivore,NA,simple,simple
X232_fail_Brown_Trout,232.fail,Seeforelle,Brown.Trout,Salmo_trutta,NA,8032,Actinopterygii,Salmoniformes,Salmonidae,⋯,ectotherm/poikilotherm,3/15/12,winter,453,Polyethylene freezer bag,gut section,carnivore,NA,simple,simple


In [8]:
# qPCR data
qPCR = Fread(qPCR_file) %>%
    filter.(grepl('Unkn-', Content)) %>%
    filter.(!grepl('marburg', Sample))
qPCR

Well,Fluor,Target,Content,Sample,Replicate,Cq,Cq.Mean,Cq.Std..Dev,Starting.Quantity..SQ.,⋯,copies.per.g,experiment,DNA.conc..ng.ul..diluted...used.for.qPCR.,Biological.Set.Name,Well.Note,class,order,family,genus,scientific_name
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<date>,<dbl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>
A04,SYBR,Methanothermobacter,Unkn-1,X13_Wolf,1,NA,0,0,NA,⋯,0,2021-03-31,NA,NA,NA,Mammalia,Carnivora,Canidae,Canis,Canis lupus
A05,SYBR,Methanothermobacter,Unkn-1,X13_Wolf,2,NA,0,0,NA,⋯,0,2021-03-31,NA,NA,NA,Mammalia,Carnivora,Canidae,Canis,Canis lupus
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
H05,SYBR,Methanothermobacter,Unkn-09,X18_West_European_Hedgehog,2,NA,0,0,NA,⋯,0,2021-05-11,1.67,NA,NA,Mammalia,Insectivora,Erinaceidae,Erinaceus,Erinaceus europaeus
H06,SYBR,Methanothermobacter,Unkn-09,X18_West_European_Hedgehog,3,NA,0,0,NA,⋯,0,2021-05-11,1.67,NA,NA,Mammalia,Insectivora,Erinaceidae,Erinaceus,Erinaceus europaeus


In [9]:
# host tree
host_tree = ape::read.tree(host_tree_file)
host_tree


Phylogenetic tree with 469 tips and 327 internal nodes.

Tip labels:
  X63_fail1_European_Perch, X63_fail2_European_Perch, X64_Eurasian_Ruffe, X82_Eurasian_Ruffe, X82_fail_Eurasian_Ruffe, X255_Zander, ...
Node labels:
  , @_22_@, @_8_@, @_4_@, @_3_@, @_1_@, ...

Rooted; includes branch lengths.

## Overlap

In [13]:
overlap(qPCR$Sample, meta$SampleID)

intersect(x,y): 46 
setdiff(x,y): 0 
setdiff(y,x): 429 
union(x,y): 475 


In [10]:
overlap(qPCR$Sample, host_tree$tip.label)

intersect(x,y): 46 
setdiff(x,y): 0 
setdiff(y,x): 423 
union(x,y): 469 


# Host tree

## Filtering

In [11]:
to_rm = setdiff(host_tree$tip.label, qPCR$Sample)
host_tree_f = ape::drop.tip(host_tree, to_rm)
host_tree_f


Phylogenetic tree with 46 tips and 44 internal nodes.

Tip labels:
  X372_Emu, X33_Wild_Turkey, X272_Black_Grouse, X131_Rock_Ptarmigan, X270_Common_Pheasant, X271_Common_Pheasant, ...
Node labels:
  @_196_@, @_89_@, @_88_@, @_46_@, @_40_@, @_38_@, ...

Rooted; includes branch lengths.

## Writing host tree

In [8]:
F = file.path(work_dir, 'unified_metadata_complete_20190416_IndD_qPCR.nwk')
write.tree(host_tree_f, F)
cat('File written:', F, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR//unified_metadata_complete_20190416_IndD_qPCR.nwk 


# Formatting for iTOL

## Copies per g feces

In [9]:
# formatting
df = qPCR %>%
    select.(Sample, copies.per.g) %>%
    mutate.(copies.per.g = log10(copies.per.g)) %>%
    mutate.(copies.per.g = ifelse(is.infinite(copies.per.g), 0, copies.per.g)) %>%
    summarize.(minimum = min(copies.per.g, na.rm=TRUE),
               q1 = quantile(copies.per.g, 0.25, na.rm=TRUE),
               median = median(copies.per.g, na.rm=TRUE),
               q3 = quantile(copies.per.g, 0.75, na.rm=TRUE),
               maximum = max(copies.per.g, na.rm=TRUE),
               .by=c('Sample')) %>%
    as.data.frame
rownames(df) = df$Sample
df$Sample = NULL
df

,minimum,q1,median,q3,maximum
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X13_Wolf,0.000000,0.000000,0.000000,0.00000,0.00000
X290_Eastern_Grey_Kangaroo,5.028918,5.122214,5.215509,5.24846,5.28141
⋮,⋮,⋮,⋮,⋮,⋮
X7_European_Otter,0,0,0,0,0
X18_West_European_Hedgehog,0,0,0,0,0


In [10]:
df %>%
    filter.(!is.infinite(median)) %>%
    .$median %>% summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   0.000   0.000   1.710   3.885   6.492 

In [11]:
# writing
F = file.path(work_dir, 'unified_metadata_complete_20190416_IndD_qPCR-htmp.txt')
itol_boxplot(df, dataset_label = 'copes_per_g', out_file=F)

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR//unified_metadata_complete_20190416_IndD_qPCR-htmp.txt 


## Input gDNA

### color strip

In [12]:
qPCR_f = qPCR %>%
    filter.(!grepl('marburg', Sample)) %>%
    filter.(!is.na(copies.per.g)) %>%
    mutate.(input_5ng = ng.per.reaction < 5 %>% as.character) %>%
    distinct.(Sample, input_5ng) %>%
    as.data.frame
rownames(qPCR_f) = qPCR_f$Sample
qPCR_f$Sample = NULL
qPCR_f

,input_5ng
,<lgl>
X13_Wolf,FALSE
X290_Eastern_Grey_Kangaroo,FALSE
⋮,⋮
X7_European_Otter,FALSE
X18_West_European_Hedgehog,FALSE


In [13]:
# custom legend
legend = data.frame('labels' = c('TRUE', 'FALSE'),
                    'colors' = c('#999999', '#33cc33'),
                    'shapes' = 1:2)
legend

labels,colors,shapes
<chr>,<chr>,<int>
TRUE,#999999,1
FALSE,#33cc33,2


In [14]:
# writing
F = file.path(work_dir, 'unified_metadata_complete_20190416_IndD_qPCR-gDNA-clrstrp.txt')
itol_colorstrip(qPCR_f, 'input_gDNA_gte5ng', out_file=F, legend=legend)

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR//unified_metadata_complete_20190416_IndD_qPCR-gDNA-clrstrp.txt 


### heatmap

In [15]:
qPCR_f = qPCR %>%
    filter.(!grepl('marburg', Sample)) %>%
    filter.(!is.na(copies.per.g)) %>%
    summarize.(ng.per.reaction = mean(ng.per.reaction),
               .by=c(Sample)) %>%
    as.data.frame
rownames(qPCR_f) = qPCR_f$Sample
qPCR_f$Sample = NULL
qPCR_f

,ng.per.reaction
,<dbl>
X13_Wolf,5
X290_Eastern_Grey_Kangaroo,5
⋮,⋮
X7_European_Otter,5
X18_West_European_Hedgehog,5


In [16]:
# writing
F = file.path(work_dir, 'unified_metadata_complete_20190416_IndD_qPCR-gDNA-htmp.txt')
itol_heatmap(qPCR_f, 'input_gDNA_ng', out_file=F, dist_method=NULL)

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR//unified_metadata_complete_20190416_IndD_qPCR-gDNA-htmp.txt 


## Host diet

In [17]:
meta_f = meta %>%
    semi_join(qPCR, c('SampleID'='Sample'))
meta_f

SampleID,host_subject_id,host_german_name,host_common_name,scientific_name,scientific_name_subspecies,host_taxid,class,order,family,⋯,temp_regulation,collection_date,season,samp_store_days,samp_collect_device,sampling_method,diet,feeding_style,digestive_system,digestive_system_detail
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
X1_Eurasian_Beaver,1,Biber,Eurasian.Beaver,Castor_fiber,NA,10185,Mammalia,Rodentia,Castoridae,⋯,endotherm/homoiotherm,5/5/11,spring,141,Screw cap vial,feces sampling,herbivore,caecotroph,hindgut,caecum
X7_European_Otter,7,Fischotter,European.Otter,Lutra_lutra,NA,9657,Mammalia,Carnivora,Mustelidae,⋯,endotherm/homoiotherm,7/25/11,summer,61,Screw cap vial,feces sampling,carnivore,NA,simple,simple
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X110_fail_Noctule,110.fail,Grosser Abendsegler,Noctule,Nyctalus_noctula,NA,51300,Mammalia,Chiroptera,Vespertilionidae,⋯,endotherm/homoiotherm,1/18/12,winter,6,Screw cap vial,gut section,carnivore,NA,simple,simple
X112_fail_Eurasian_Sparrowhawk,112.fail,Sperber,Eurasian.Sparrowhawk,Accipiter_nisus,NA,211598,Aves,Falconiformes,Accipitridae,⋯,endotherm/homoiotherm,1/18/12,winter,6,Screw cap vial,gut section,carnivore,NA,simple,simple


In [32]:
# output file
out_file = file.path(work_dir, 'unified_metadata_complete_20190416_IndD_diet_itol-clrstrp.txt')
out_file

[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR//unified_metadata_complete_20190416_IndD_diet_itol-clrstrp.txt"

In [35]:
# diet
df = meta_f %>%
    select.(SampleID, diet) %>%
    as.data.frame
rownames(df) = df$SampleID
df$SampleID = NULL

# legend
y = meta %>% pull.(diet) %>% unique
print(y)
cols = c('#FFCC33', '#00CC00', '#FF0000')
legend = data.frame(labels = y,
                    colors = cols,
                    shapes = rep(1, length(labels)))

# strip
itol_colorstrip(df, 'diet', out_file, legend=legend)

[1] "omnivore"  "herbivore" "carnivore"
File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR//unified_metadata_complete_20190416_IndD_diet_itol-clrstrp.txt 


## Host body temp

In [38]:
bt = Fread(bt_file) %>%
    mutate.(SampleID = gsub('^F', 'XF', SampleID))
bt

SampleID,class,order,family,genus,scientific_name,Tb,dataset
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
XF14_Common_Bream,Actinopterygii,Cypriniformes,Cyprinidae,Abramis,Abramis brama,NA,NA
XF35_Red_Deer,Mammalia,Artiodactyla,Cervidae,Cervus,Cervus elaphus,38.6,WalzerXXXX
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,Reptilia,Crocodilia,Crocodylidae,Crocodylus,Crocodylus niloticus,NA,NA
X67_European_Roe,Mammalia,Artiodactyla,Cervidae,Capreolus,Capreolus capreolus,40.4,WalzerXXXX


In [42]:
# filtering
bt_f = bt %>%
    semi_join(qPCR, c('SampleID'='Sample'))
bt_f

SampleID,class,order,family,genus,scientific_name,Tb,dataset
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
X117_Long_eared_Owl,Aves,Strigiformes,Strigidae,Asio,Asio otus,40.1,Clarke2014
X119_Ural_Owl,Aves,Strigiformes,Strigidae,Strix,Strix uralensis,40.2,Prinzinger1990
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X417_Giraffe,Mammalia,Artiodactyla,Giraffidae,Giraffa,Giraffa camelopardalis,37.7,WalzerXXXX
X7_European_Otter,Mammalia,Carnivora,Mustelidae,Lutra,Lutra lutra,38.1,Clarke2010


In [43]:
# output file
out_file = file.path(work_dir, 'unified_metadata_complete_20190416_IndD_bt_itol-htmp.txt')
out_file

[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR//unified_metadata_complete_20190416_IndD_bt_itol-htmp.txt"

In [48]:
# format
df = bt_f %>%
    select.(SampleID, Tb) %>%
    as.data.frame
rownames(df) = df$SampleID
df$SampleID = NULL

itol_heatmap(df, 'bt', out_file, dist_method = NULL)

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/qPCR/174FR//unified_metadata_complete_20190416_IndD_bt_itol-htmp.txt 


# sessionInfo

In [17]:
sessionInfo()

R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq2/lib/libopenblasp-r0.3.10.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.9 ape_5.4-1         tidytable_0.5.8   data.table_1.13.6
[5] ggplot2_3.3.3     tidyr_1.1.2       dplyr_1.0.3      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.6        pillar_1.4.7      compiler_4.0.3    base64enc_0.1-3  
 [5] tools_4.0.3       digest_0